**Table of contents**

- [Propositional logic](#propositional-logic-an-introduction)
    - [Truth tables](#truth-tables)  
    - [Knowledge base (KB)](#knowledge-base-kb)
        - [Entailment](#entailment)
        - [Satisfiability](#satisfiability)
    - [Inference: model checking](#inference-model-checking)
        - [Model checking: properties](#model-checking-properties)
- [Symbolic maths `sympy`](#symbolic-maths-sympy)
    - [Symbolic variables](#symbolic-variables)
    - [Some examples of symbolic math](#some-examples-of-symbolic-math)
    - [`sympy` and logic operators](#sympy-and-logic-operators)
    - [Truth tables in `sympy`](#truth-tables-in-sympy)
- [Model checking: using `sympy`](#model-checking-using-sympy)
- [Normal forms for logical expressions](#normal-forms-for-logical-expressions)
    - [De Morgan's law](#de-morgans-law) 
    - [Conjunctive normal form (CNF)](#conjunctive-normal-form-cnf) 
    - [Negation normal form (NNF)](#negation-normal-form-nnf)
    - [CNF and NNF using `sympy`](#cnf-and-nnf-using-sympy)
- [Inference with propositional logic](#inference-with-propositional-logic)
    - [Main inference rules](#main-inference-rules)
    - [Inference by resolution](#inference-by-resolution)
    - [Horn clauses](#horn-clauses)
    - [How to manage KB in `sympy`](#how-to-manage-kb-in-sympy)
    - [Forward chaining](#forward-chaining)  
    - [Backward chaining](#backward-chaining)  
- [External references](#external-references)

# Propositional logic: an introduction

Logic is the formal study of valid reasoning. It deals with the rules that govern how propositions (statements) can be combined to deduce correct conclusions from given premises.

In AI, logic is used to represent knowledge and draw inferences, especially in symbolic systems such as automated reasoning or inference engines.

Many logic implementation exists, such as:
* Boolean logic
* Fuzzy logic
* Propositional logic (that we will discuss here)
* Predicative logic (Prolog)

The main characters of the propositional logic are *atomic sentences* and *complex sentences*. The atomic sentences are that kind of expressions that express just a truth, so their state could be **true** or **false**. We could have $N$ atomic sentences that express a fact. Then, we can combine two or more atomic sentences in order to get a **complex sentences**, the operators that we have to combine theese sentences are listed below:

* **AND** "$\wedge$": the sentence is true only if both atomic sentences are true
* **OR** "$\vee$": the sentence is true if at least one atomic sentence is true
* **NOT** "$\neg$": this operator negate the state, for instance, if $P=True$, then $\neg P=False$, and viceversa
* **IMPLIES** "$\Rightarrow$": given $P \Rightarrow Q$, this operator return true, if $P=True$ and $Q=False$
* **IF AND ONLY IF (bidirectional)** "$\Leftrightarrow$": bidirectional implication, return true, if both atomic sentences have the same state

## Truth tables

Like the *boolean logic*, also the operators of the propositional logic have their truth tables, actually also boolean logic is a sort of propositional logic.

Just to remind, the truth table is a tool that describe the output result for any possible combination of input for a specific logical operator.

[**CONJUCTION** (`AND`)]

| $P$ | $Q$ | $P \wedge Q$ |
|:-:|:-:|:-:|
| `true` | `true` | `true` |
| `true` | `false` | `false` |
| `false` | `true` | `false` |
| `false` | `false` | `false` |

[**DISJUNCTION** (`OR`)]

| $P$ | $Q$ | $P \vee Q$ |
|:-:|:-:|:-:|
| `true` | `true` | `true` |
| `true` | `false` | `true` |
| `false` | `true` | `true` |
| `false` | `false` | `false` |

[**NEGATION** (`NOT`)]

| $P$ | $\neg P$ |
|:-:|:-:|
| `true` | `false` |
| `false` | `true` |

[**IMPLICATION** (`IMPLIES`)]

| $P$ | $Q$ | $P \Rightarrow Q$ |
|:-:|:-:|:-:|
| `true` | `true` | `true` |
| `true` | `false` | `false` |
| `false` | `true` | `true` |
| `false` | `false` | `true` |

[**BIDIRECTIONAL** (`IF AND ONLY IF`)]

| $P$ | $Q$ | $P \Leftrightarrow Q$ |
|:-:|:-:|:-:|
| `true` | `true` | `true` |
| `true` | `false` | `false` |
| `false` | `true` | `false` |
| `false` | `false` | `true` |

## Knowledge base (KB)

The knownledge base is a concept not strictly correlated to propositional logic, is a more general concept in logic fields and also propositional logic use it. Basically, the knowledge base, usually denoted with **KB**, is a set of sentences that define the rules of the environment, to be clear is a set that contains all sentences that the agent knows to be true. The KB is essential to apply inference on our sentences, the whole inferential process is based on KB, without it we would not able to get a conclusion starting from two sentences.

An example:

```bash
# Our atomic sentences
P = is a student?
Q = can access to the library!

# Our KB contains only one rule, IF P IS A STUDENT (P=True) THEN P CAN ACCESS TO THE LIBRARY (Q=True)
KB = {P -> Q}
```

From the the KB we know that all students can access to the library!


### Entailment

In propositional logic, entailment (or logical consequence), denoted by $\models$, is a fundamental relationship between a set of premises and a conclusion. A conclusion is a logical consequence of a set of premises if whenever the premises are true, the conclusion must also be true.

Formally:

$$
\alpha \models \beta
$$

This means there is no possible situation where all premises $\alpha$ are true and $\beta$ is false simultanesly. The truth of the premises gurantees the truth of the conclusion.

### Satisfiability

In propositional logic, a formula is satisfiable if there exists at least one truth assignment to its variables that makes the formula true.
In other words, a formula is satisfiable if it can be true in at least one model.

For example,

$P \vee Q$

is a satisfiable query, because it is true whenever at least on of $P$ or $Q$ is true.

Instead,

$ P \wedge \neg P $

is not satisfiable, beacause cannot exist a model where $P$ is simoultaniesly true and false.


In [1]:
from sympy.logic import satisfiable, And, Or, Not
from sympy import symbols

P, Q = symbols('P, Q')

expression_1 = Or(P, Q)            # satisfiable expression
expression_2 = And(P, Not(P))      # not satisfiable expression

# Output
print(f"P ∨ Q: {satisfiable(expression_1)}")
print(f"P ∧ ¬P: {satisfiable(expression_2)}")

P ∨ Q: {Q: True, P: True}
P ∧ ¬P: False


## Inference: model checking

Inference denotes the process of deriving new statements (or conclusions) from existing ones. AI agents can use an inference algorithm to determine whether a given query sentence $\alpha$ is true or false.

An example

```bash
# Atomic sentences
P: it is a Tuesday
Q: It is raining

# The sentence that we wanna infer
R: Harry will go for a run

# Knowledge base: If is Tuesday and is NOT Raining, then R is True (Harry will go for a run)
KB: (P AND NOT(Q)) -> R
```

To infer $R$, we need to use an inference algorithm that answers the central question of entailment. Given a query sentence $\alpha$ about the world, does $KB \models \alpha$? In other words, 'Can we conclude that sentence $\alpha$ is true based solely on the information in our KB?

There are many algorithm to do this, the most simple is **model checking**, that appplies the following steps:

1. Enumerate all possible models for sentences $KB \cup \{\alpha\}$
1. Check if $\alpha$ is true in every model in which $KB$ is true
    - If so, $KB \models \alpha$
    - Otherwise, $KB$ does not entail $\alpha$

To proceed, we have to implement the truth tables

<table style="border-collapse:collapse;border-color:#93a1a1;border-spacing:0" class="tg"><thead><tr><th style="background-color:#657b83;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:Arial, sans-serif;font-size:14px;font-weight:bold;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal" colspan="3">Atomic sentences</th><th style="background-color:#657b83;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:Arial, sans-serif;font-size:14px;font-weight:bold;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal" colspan="3">Premises</th><th style="background-color:#657b83;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:Arial, sans-serif;font-size:14px;font-weight:bold;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Conclusion</th></tr></thead>
<tbody>
<tr><td style="background-color:#eee8d5;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">P</td><td style="background-color:#eee8d5;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Q</td><td style="background-color:#eee8d5;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">R</td>
<td style="background-color:#eee8d5;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">P<span style="font-style:normal">∨</span>Q</td><td style="background-color:#eee8d5;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal"><span style="font-weight:400;font-style:normal">P</span><span style="font-weight:700;font-style:normal">⇒</span><span style="font-style:normal">Q</span></td><td style="background-color:#eee8d5;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Q<span style="font-weight:700">⇒</span>R</td>
<td style="background-color:#eee8d5;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal"><span style="font-weight:400;font-style:normal">P</span>∨R</td></tr>
<tr><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td>
<td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td>
<td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td></tr>
<tr><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td>
<td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td>
<td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td></tr>
<tr><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td>
<td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td>
<td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td></tr>
<tr><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td>
<td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td>
<td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td></tr>
<tr><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td>
<td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td>
<td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td></tr>
<tr><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td>
<td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td>
<td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td></tr>
<tr><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td>
<td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td><td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">false</td>
<td style="background-color:#fdf6e3;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td></tr>
<tr><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td>
<td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td><td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td>
<td style="background-color:#ffcc67;border-color:inherit;border-style:solid;border-width:1px;color:#343434;font-family:"Courier New", Courier, monospace !important;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">true</td></tr>
</tbody></table>

The truth table suggest us that  $KB \models \alpha$, because as you can see, the conclusion is `true` in every model in which the premises are `true` (the orange highlighted rows)

### Model checking: properties

* **Soundness**: It directly implements the definition of entailment
* **Completeness**: It works for any (finite) $KB$ and $\alpha$, and the corresponding set of models is finite
* **Complexity**: $O(n^2)$, where $n$ is the number of proposition symbols appearing in $KB$ and $\alpha$. For this reason, this algorithm is infeasible when the number of proposition is high

# Symbolic maths (`sympy`)

`SymPy` (Symbolic Python) is an open-source library for symbolic mathematics. Unlike numerical libraries such as `NumPy` or `SciPy`, `SymPy` works with symbolic expressions — as if it were a computer algebra system.

Basically, in python we are not able to represent math symbols like this "$\sqrt(2)$", so `sympy` solve this problem, and how we will see, we can use it also for propositional operators representation in a symbolic way.

`sympy` offer a huge numbers of tools, but we are interest just to know the main aspects of this module.

### Symbolic variables

As suggested by the name, the symbolic math uses symbols, then we can consider the symbols the main entity in this context. Obviously `sympy` allow us to make symbols (variables). 

In [2]:
from sympy import symbols, abc

# We can define a symbol using the constructor method symbols
A = symbols("A")
display("Symbol defined by the constructor",A)

# But we can also use a set of predefined symbols importing abc from sympy
B = abc.B
Z = abc.Z
display("Suymbols predefined by the module",B,Z)


'Symbol defined by the constructor'

A

'Suymbols predefined by the module'

B

Z

### Some examples of symbolic math

In [3]:
from sympy import oo, pi, sqrt

x = symbols('x')

display(oo)
display(pi)
display(sqrt(x**2))

oo

pi

sqrt(x**2)

This module is not just for visualization, indeed it provides us a large quantity of methods to solve our equations, in practice it offers set of tools in order to apply math operations using symbolic math. This make very comfortable some difficult computation using the classical numerical math, such as the derivates.

In [4]:
from sympy import symbols, diff, sqrt, expand, simplify, init_printing

# Pretty printing for math expressions
init_printing()

# Define symbolic variables
x, y = symbols('x y')

# === 1. Derivative: diff ===
# Compute the derivative of f(x) = x^2 + 3x + 2 with respect to x
f = x**2 + 3*x + 2
df_dx = diff(f, x)
print("1. Derivative of f(x) = x^2 + 3x + 2 with respect to x:")
display(df_dx)
print()

# === 2. Square root: sqrt ===
# Create an expression with a square root
g = sqrt(x**2 + 1)
print("2. Square root of (x^2 + 1):")
display(g)
print()

# === 3. Expansion: expand ===
# Expand the expression (x + 1)^2
h = expand((x + 1)**2)
print("3. Expansion of (x + 1)^2:")
display(h)
print()

# === 4. Simplification: simplify ===
# Simplify a rational expression
expr = (x**2 - 1)/(x - 1)
simplified_expr = simplify(expr)
print("4. Simplification of (x^2 - 1)/(x - 1):")
display(simplified_expr)

1. Derivative of f(x) = x^2 + 3x + 2 with respect to x:



2. Square root of (x^2 + 1):



3. Expansion of (x + 1)^2:



4. Simplification of (x^2 - 1)/(x - 1):


### `sympy` and logic operators

`sympy` offers also tools to implements algorithms based on propositional logic, in fact we can apply the logical operators studied until now. Then we will see more sofisticated use of `sympy` for the propositional logic.

In [5]:
from sympy import And, Or, Not, Implies, Equivalent, abc

display(Not(abc.A))
display(And(abc.A, abc.B))
display(Or(abc.A, abc.B))
display(Implies(abc.A,abc.B))
display(Equivalent(abc.A,abc.B))

# But you can also use unary operators to reach an equivalent result
display(~abc.A)          # NOT
display(abc.A & abc.B)   # AND
display(abc.A | abc.B)   # OR
display(abc.A >> abc.B)  # IMPLIES 

### Truth tables in `sympy`

`sympy` allows us to compute truth tables of the logic operators automatically.

In [6]:
from sympy import symbols
from sympy.logic.boolalg import And, Or, Not, Implies, Equivalent, truth_table

def print_truth_table(truth_table=truth_table ,title=None):
    print(title)
    for combination in truth_table:
        print(combination)

A, B = symbols('A, B')

# Logic operators
AND_expression = And(A,B)
OR_expression = Or(A, B)
NOT_expression = Not(A)
BIDIRECTIONAL_expression = Equivalent(A,B)
COMPLEX_expression = Implies(A, And(A, Not(B)))

# Truth tables
and_tt = truth_table(AND_expression, [A, B])
or_tt = truth_table(OR_expression, [A,B])
not_tt = truth_table(NOT_expression, [A])
bidirectional_tt = truth_table(BIDIRECTIONAL_expression, [A,B])
complex_tt = truth_table(COMPLEX_expression, [A,B])

# Truth tables output
print_truth_table(and_tt, "\nA ∧ B: truth table")
print_truth_table(not_tt, "\n¬A: truth table")
print_truth_table(or_tt, "\nA ∨ B: truth table")
print_truth_table(bidirectional_tt, "\nA ⇔ B: truth table")
print_truth_table(complex_tt, "\nA ⇒ A ∧ ¬B: truth table")


A ∧ B: truth table
([0, 0], False)
([0, 1], False)
([1, 0], False)
([1, 1], True)

¬A: truth table
([0], True)
([1], False)

A ∨ B: truth table
([0, 0], False)
([0, 1], True)
([1, 0], True)
([1, 1], True)

A ⇔ B: truth table
([0, 0], True)
([0, 1], False)
([1, 0], False)
([1, 1], True)

A ⇒ A ∧ ¬B: truth table
([0, 0], True)
([0, 1], True)
([1, 0], True)
([1, 1], False)


## Model checking: using `sympy`

In this section we will see how to implement model checking using `sympy` module, so we we want entail a query $\alpha$ respect to the knowledge base $KB$.

We have the following rules (KB):
• If it didn’t rain, Harry visited Hagrid today
• Harry visited Hagrid or Dumbledore today, but not both
• Harry visited Dumbledore today

And theese are the facts:
• Harry did not visit Hagrid today
• It rained today

Using only theese information we can infer some information using model checking.

⚠️: In this example we will assume that Harry visited Doumbledore. In fact we put "doumbledore" in the KB, you can try to change this fact with "hagrid".

In [7]:
from sympy import symbols, And, Not, Or, Implies, satisfiable
from sympy.logic.inference import entails

rain, hagrid, dumbledore = symbols("rain hagrid dumbledore")

KB = And(
    Implies(Not(rain), hagrid),
    Or(hagrid, dumbledore),
    Not(And(hagrid, dumbledore)),
    dumbledore  # try to change this fact with "hagrid"
)

# Model checking
def model_checking(knowledge_base, query, symbols, model):
    if not symbols:
        if knowledge_base.subs(model):
            return query.subs(model)
        return True
    else:    
        remaining = symbols.copy()
        p = remaining.pop()
    
        model_true = model.copy()
        model_true[p] = True
        model_false = model.copy()
        model_false[p] = False

        return (model_checking(knowledge_base, query, remaining, model_true)) and \
               (model_checking(knowledge_base, query, remaining, model_false))

# Input test
symbols_list = [rain, hagrid, dumbledore]
queries = [hagrid, dumbledore, rain, Not(rain), And(hagrid, Not(dumbledore)), Not(dumbledore)]

# Entailment with MODEL CHECKING
print("MODEL CHECKING RESULTS")
for query in queries:
    model = {}
    result = model_checking(KB, query, symbols_list, model)
    print(f"Query test [{query}]: {result}")

# Entailment with sympy entails()
print("\nSYMPY ENTAILS RESULTS")
for query in queries:
    result = entails(query, [KB])
    print(f"Query test [{query}]: {result}")

# Satisfiability with sympy
print(f'\nSATISFIABILITY\n{satisfiable(KB, all_models=False)}')

MODEL CHECKING RESULTS
Query test [hagrid]: False
Query test [dumbledore]: True
Query test [rain]: True
Query test [~rain]: False
Query test [hagrid & ~dumbledore]: False
Query test [~dumbledore]: False

SYMPY ENTAILS RESULTS
Query test [hagrid]: False
Query test [dumbledore]: True
Query test [rain]: True
Query test [~rain]: False
Query test [hagrid & ~dumbledore]: False
Query test [~dumbledore]: False

SATISFIABILITY
{dumbledore: True, rain: True, hagrid: False}


## Normal forms for logical expressions

Complex logical expressions can be represented in various forms, and sometimes these form changes can be very useful for different tasks.
As we will see, we have different ways to do this.

### De Morgan's law

De Morgan's Laws are two fundamental equivalences in propositional logic used to manipulate negations of compound statements:

1. $\neg(P \wedge Q) \equiv \neg P \vee \neg Q$
2. $\neg(P \vee Q) \equiv \neg P \wedge \neg Q$

These laws are crucial for:
- Simplifying logical expressions
- Transforming formulas into NNF
- Removing negations from complex expressions

These transformations preserve the **logical equivalence** of the original formula and are especially useful in automated reasoning and logic-based algorithms.

### Conjunctive normal form (CNF)

Conjunctive Normal Form (CNF) is a standardized format for logical formulas used in SAT (Boolean satisfiability problem) solving, theorem proving, and logic programming.

A formula in CNF is a **conjunction of one or more clauses**, where each **clause is a disjunction of literals**.

- **Literal**: a variable or its negation (e.g., $P$, $\neg Q$)
- **Clause**: disjunction of literals (e.g., $\neg P \vee Q \vee R$)
- **CNF Formula**: conjunction of clauses (e.g., $(\neg P \vee Q) \wedge (R \vee S \vee \neg T)$)

The manipulations applied by CNF are:

* Eliminates implications ($\Rightarrow$)
* Elminates bidirectionala ($\Leftrightarrow$)
* Push negations inward using De Morgan's law
* Uses distributive law in order to distribute disjunction $(\vee)$ wherever possible

### Negation normal form (NNF)

Negation Normal Form (NNF) is a way of rewriting logical formulas so that:
- The only allowed logical connectives are **∧ (AND)**, **∨ (OR)**, and **¬ (NOT)**.
- The **NOT operator (¬)** appears **only in front of propositional variables (atoms)**.
- There are **no implications (⇒)** or **bi-implications (⇔)**.

To convert any propositional formula into NNF, we apply:
1. Elimination of implications and bi-implications:
   - $( P \Rightarrow Q )$ becomes $( \neg P \vee Q )$
   - $( P \Leftrightarrow Q )$ becomes $( (P \wedge Q) \vee (\neg P \wedge \neg Q) )$
2. Application of De Morgan’s Laws to push negations inward.
3. Simplification of double negations: $(\neg(\neg P))$ becomes $( P )$

### CNF and NNF using `sympy`

In [8]:
from sympy.logic import to_cnf
from sympy import abc

# Atomic terms
P = abc.P
Q = abc.Q

# CNF Conversion
original_expr1 = Implies(P,Q)
original_expr2 = Not(And(P,Q))
cnf_expr1 = to_cnf(original_expr1)
cnf_expr2 = to_cnf(original_expr2)

# Output
print("Original expression 1")
display(original_expr1)
print("Expression 1 CNF version")
display(cnf_expr1)
print("Original expression 2")
display(original_expr2)
print("Expression 2 CNF version")
display(cnf_expr2)

Original expression 1


Expression 1 CNF version


Original expression 2


Expression 2 CNF version


In [9]:
from sympy.logic import to_nnf

# Atomic terms
A = abc.A
B = abc.B

# NNF conversion
original_expr = Equivalent(A,B)
nnf_expr = to_nnf(original_expr)

# Output
print("Original expression")
display(original_expr)
print("NNF converted")
display(nnf_expr)

Original expression


NNF converted


## Inference with propositional logic

There are others algorithms outside of the **model checking**, in particular we will explore the **forward chaining** and **backward chaining** algorithms. Firstly we have to introduce the general concepts of inference.

* **Logically equivalence**: we say that two sentences $\alpha$ and $\beta$ are logically equivalent (if and only if) $\alpha \equiv \beta$, if they are true under the same models, so when $\alpha \models \beta$ and $\beta \models \alpha$. An istance could be $(P \wedge Q)$ is equivalent to $(Q \wedge P)$.
* **Valid**: if a sentence is true in all models, an example is the sentence $(P \vee \neg P)$ (*tautology sentence*)
* **satisfiable (SAT)**: if a sentence is true only in some models, an example is the sentence $(P \wedge Q)$. The satisfiability can be used to proof the entailment using an approach called *reductio ad absurdum*, formally $\alpha \models \beta$ if and only if ($\alpha \wedge \neg \beta$)

A formal way to describe an inference rule is this:

$$
\frac{premises}{conclusions}
$$

### Main inference rules

The main inference rules are following reported, they are very easy to proof using `model checking` algorithm.

<table style="border-collapse:collapse;border-color:#C44D58;border-spacing:0" class="tg"><thead><tr><th style="background-color:#FE4365;border-color:#C44D58;border-style:solid;border-width:1px;color:#fdf6e3;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Inference rule</th><th style="background-color:#FE4365;border-color:#C44D58;border-style:solid;border-width:1px;color:#fdf6e3;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Example</th></tr></thead>
<tbody><tr><td style="background-color:#FFA4A0;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">And elimination</td><td style="background-color:#FFA4A0;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">P ∧ Q, P<br>_______<br>Q</td></tr>
<tr><td style="background-color:#F9CDAD;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">And introduction</td><td style="background-color:#F9CDAD;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">P, Q<br>_______<br>P ∧ Q</td></tr>
<tr><td style="background-color:#FFA4A0;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Or introduction</td><td style="background-color:#FFA4A0;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">P<br>_______<br>P ∨ Q</td></tr>
<tr><td style="background-color:#F9CDAD;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">First De Morgan's law</td><td style="background-color:#F9CDAD;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">¬(P ∧ Q)<br>_______<br>¬P ∨ ¬Q<br></td></tr>
<tr><td style="background-color:#FFA4A0;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Double negation</td><td style="background-color:#FFA4A0;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">¬(¬P)<br>_______<br>P</td></tr>
<tr><td style="background-color:#F9CDAD;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Modus ponens</td><td style="background-color:#F9CDAD;border-color:inherit;border-style:solid;border-width:1px;color:#002b36;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">P → Q, P<br>_______<br>Q</td></tr></tbody></table>

In [10]:
from sympy import And, Not, Implies, Or, Equivalent
from sympy.logic.inference import satisfiable
from sympy import symbols

P, Q = symbols("P, Q")

# And elimination
tmp_KB = And(And(P,Q), Q)
model = {}
result = model_checking(tmp_KB, Q, [P, Q], model)
print(f"Q inference [AND elimination]: {result}")

# Modus ponens
tmp_KB = And(Implies(P, Q), P)
model = {}
result = model_checking(tmp_KB, Q, [P, Q], model)
print(f"Q inference [Modus ponens]: {result}")

# Double negation
tmp_KB = And(Not(Not(P)))
model = {}
result = model_checking(tmp_KB, P, [P,Q], model)
print(f"P inference [Double negation]: {result}")

Q inference [AND elimination]: True
Q inference [Modus ponens]: True
P inference [Double negation]: True


### Inference by resolution

We have already defined that model checking is a too expansive way to proof some entailment, for this reason other algorithms are implemented, in particular, in this section we will explore `inference by resolution`.

We can use inference by resolution to determine if $KB \models \alpha$ just checking if:

$$
\text{satisfiable}(\text{CNF}(KB \wedge \alpha)) = False
$$

if the above expression is not satisfiable (a contraddiction occurs), then we can say that $KB \models \alpha$.

We can try to use `sympy` to implements inference by resolution. Consider the following knowledge base:

$$
(A \vee B) \wedge (\neg B \vee C) \wedge (\neg C)
$$

We want check if:

$$
KB \models A
$$

⚠️: Inference by resolution is already implemented also in the KB_class.

Below the `sympy` implementation that solves this query using inference by resolution.

In [11]:
from sympy import symbols
from sympy.logic.boolalg import Implies, Not, to_cnf, And, Or
from sympy.logic.inference import satisfiable

A, B, C = symbols('A B C')

kb = And(Or(A,B),Or(Not(B),C),Not(C))

negated_query = Not(A)

full_expr = And(kb, negated_query)

cnf_expr = to_cnf(full_expr, simplify=True)

print("Inference by resolution:")
satisfiable(cnf_expr)

Inference by resolution:


False

Following, another more intuitive example.

$$
KB = \text{Ron is in the great hall} \vee \text{Hermione is in the library}, \neg(\text{Ron is in the great hall})
$$

$$
Query = \text{Hermione is in the library}
$$

As said before, we have to check if:

$$
\text{satisfiable}(\text{CNF}(KB \wedge \neg(\text{Hermione is in the library})))
$$

In [12]:
from sympy import symbols
from sympy.logic.boolalg import Implies, Not, to_cnf, And, Or
from sympy.logic.inference import satisfiable

ron_is_in_the_great_hall = symbols('ron_is_in_the_great_hall')
hermione_is_in_the_library = symbols('hermione_is_in_the_library') 

kb = And(Or(ron_is_in_the_great_hall, hermione_is_in_the_library), Not(ron_is_in_the_great_hall))

negated_query = Not(hermione_is_in_the_library)

full_expr = And(kb, negated_query)

cnf_expr = to_cnf(full_expr, simplify=True)

print("Inference by resolution result:")
satisfiable(cnf_expr)

Inference by resolution result:


False

### Horn clauses

A horn clause is a special case of disjunction of literals where at most one of them must be true.

An example of horn clause is:

$$
\neg P \vee Q \vee \neg S
$$

In addition, a horn clause can be expressed as an implication:

$$
\neg P \vee \neg Q \vee S
$$

become

$$
P \wedge Q \Rightarrow S
$$

The horn clauses are useful in many application, in particular, for computational costs reduction when we have to proof a sentence.

In [13]:
"""
    Truth tables of Horn clause and its implication form.
    This implementation proof the equivalence of the horn and its implication form
"""

from sympy.logic.boolalg import truth_table

P,Q,S = symbols("P,Q,S")

# Horn clause
horn = Or(Not(P), Not(Q), S)

# Horn as implication
horn_as_implication = Implies(And(P, Q), S)

# Truth tables
horn_tt = truth_table(horn, [P,Q,S])
horn_as_implication_tt = truth_table(horn_as_implication, [P,Q,S])

print_truth_table(horn_tt, "Horn truth table\n")
print_truth_table(horn_as_implication_tt, "\nHorn as implication truth table\n")

Horn truth table

([0, 0, 0], True)
([0, 0, 1], True)
([0, 1, 0], True)
([0, 1, 1], True)
([1, 0, 0], True)
([1, 0, 1], True)
([1, 1, 0], False)
([1, 1, 1], True)

Horn as implication truth table

([0, 0, 0], True)
([0, 0, 1], True)
([0, 1, 0], True)
([0, 1, 1], True)
([1, 0, 0], True)
([1, 0, 1], True)
([1, 1, 0], False)
([1, 1, 1], True)


We can also implement two functions which check if a clause is a horn clause, while the other function can transform a horn clause as implication.

In [14]:
from sympy import Symbol

""" 
    Function that check if a clause is a horn clause

    @param
        clause (sympy clause): clause defined using sympy module
"""
from sympy import Or, Not, Implies, And, Symbol, false

def check_horn(clause):
    terms = clause.args

    # Check if a fact is positive
    if isinstance(clause, Symbol):
        return True

    # Check for pure clauses (disdisjunction)
    if isinstance(clause, Or):    
        positive_atoms = 0
        for term in terms:
            if not isinstance(term, Not):
                positive_atoms += 1
        if positive_atoms > 1:
            return False
        return True
    
    # Check for clauses as implications
    if isinstance(clause, Implies):
        conclusion = clause.args[1]
        premisees = clause.args[0]

        if not isinstance(conclusion, Symbol) and conclusion != false:
            return False

        if isinstance(premisees, Symbol):
            return True

        if isinstance(premisees, And):
            for literal in premisees.args:
                if not isinstance(literal, Symbol):
                    return False
            return True
        
        return False

    return False 



"""
    Function that converts a horn clause as implication
    
    @param
        clause (sympy clause): Horn clause defined using sympy module
"""
def horn_to_implication(clause):
    # Check if a clause is a pure horn clause
    if check_horn(clause) and isinstance(clause, Or):
        terms = clause.args
        negative_terms = []
        positive_term = None
        for term in terms:
            if isinstance(term, Not):
                negative_terms.append(term.args[0])
            else:
                positive_term = term

        if positive_term is None:
            return clause

        return Implies(And(*negative_terms), positive_term)
    else:
        return clause
    
# Usage
P, Q, M, L = symbols("P Q M L")

clause1 = Or(Not(P), Not(Q), M, Not(L))   # Horn clause
clause2 = Or(Not(P), Q, Not(M), L)        # No-Horn clause
clause3 = Implies(And(P,Q,L),M)
print(f"Is clause 1 a horn clause?: {check_horn(clause1)}")
print(f"Is clause 2 a horn clause?: {check_horn(clause2)}")

print("\nHorn clause 1")
display(clause1)
print("Horn clause 1 as implication")
display(horn_to_implication(clause1))

Is clause 1 a horn clause?: True
Is clause 2 a horn clause?: False

Horn clause 1


Horn clause 1 as implication


### How to manage KB in `sympy`

In this section we will try to implement a class useful to manage our knowledge base easily, basically, we want implement a class which contains all studied concepts that we have seen until now.

ℹ️: If you want to use this class in a python source code, replace all `display()` functions with a classical `print()`.

In [15]:
from sympy.logic import And, Or, Not, Implies
from sympy import symbols, Symbol, to_cnf
from sympy.logic.inference import satisfiable

"""
    Class that defines a Knowledge base, in order to manage it easily.
    
    @params:
        symbols_list (list): List of sympy.Symbol objects
        *clauses (Any): Each arguments will be handle as a separate clause
"""
class KB_class:
    def __init__(self, symbols_list, *clauses):
        self.knowledge_base = [clause for clause in clauses]
        self.symbols = [symbol for symbol in symbols_list]
    
    """
        Add one or more clauses to KB
    """
    def add_clauses(self, *clauses):
        for clause in clauses:
            self.knowledge_base.append(clause)

    """
        Remove one or more clauses from the KB
    """
    def remove_clauses(self, *clauses):
        self.knowledge_base = [clause for clause in self.knowledge_base if clause not in clauses]

    """
        Add one or more symbols 
    """
    def add_symbols(self, *symbols_list):
        for symbol in symbols_list:
            if isinstance(symbol, Symbol):
                self.symbols.append(symbol)
            else:
                print("Error: symbols must be sympy.Symbol object")
    
    """
        Remove one or more symbols, also the clauses which contain the removed symbols will be removed.
        sympy.Symbol.atoms() is used to extract the involved symbols in a clause
    """
    def remove_symbols(self, *symbols_list):
        self.symbols = [symbol for symbol in self.symbols if symbol not in symbols_list]
        for clause in self.knowledge_base:
            atoms = clause.atoms()
            for symbol in symbols_list:
                if symbol in atoms:
                    self.remove_clauses(clause)

    """
        Check if the whole knowlwdge base is composed by horn clauses
    """
    def is_horn(self):
        no_horn = 0
        
        for clause in self.knowledge_base:
            if not check_horn(clause):
                no_horn += 1
        if no_horn > 0:
            return False
        return True

    """
        Replace all horn clauses with their respective implication
    """
    def horn_to_implication(self):
        for clause in self.knowledge_base:
            if check_horn(clause) and isinstance(clause, Or):
                self.add_clauses(horn_to_implication(clause))
                self.remove_clauses(clause)
    
    """
        Returns the known facts in the knowledge base
    """
    def extract_facts(self):
        facts = []
        for clause in self.knowledge_base:
                if len(clause.atoms()) == 1:
                    facts.append(clause)
        return facts

    """
        Print all knowledge base facts/rules
    """
    def print_clauses(self):
        for clause in self.knowledge_base:
            display(clause)

    """
        Print all knowledge base symbols
    """
    def print_symbols(self):
        display(self.symbols)

    """
        Check if a sentence contained into the KB
    """
    def contains(self, clause):
        if clause not in self.knowledge_base:
            return False
        return True

    """
        Return KB as a unique AND
    """
    def build_KB(self):
        return And(*self.knowledge_base)
    
    """
        Check if KB entail a query alpha (KB:=a) using inference by resolution
    """
    def entails(self, query):
        kb_as_and = self.build_KB()
        join_expression = And(kb_as_and, Not(query))
        join_expression = to_cnf(join_expression, simplify=True)
        if not satisfiable(join_expression):
           return True
        return False

Now, we can try our class.

In [16]:
# Define symbols
P, Q, A, B = symbols("P Q A B")

# Instanciate Knowledge base and print their properties
knowledge_base = KB_class([P, Q, A, B],And(P,Q), Not(Q), Implies(B,A))
print("Symbols")
knowledge_base.print_symbols()
print("\nKnowledge Base")
knowledge_base.print_clauses()

# Trying some methods 
knowledge_base.add_clauses(Implies(Q,B), Not(A))
knowledge_base.remove_symbols(P)

# Output after methods
print("------ AFTER METHODS -----")
print("\nSymbols")
knowledge_base.print_symbols()
print("\nKnowledge Base")
knowledge_base.print_clauses()

# Try entailment
print("\nCan KB entails A?")
print(knowledge_base.entails(A))

Symbols



Knowledge Base


------ AFTER METHODS -----

Symbols



Knowledge Base



Can KB entails A?
False


Others usage on horn KB

In [17]:
KB_horn = KB_class(
    [P, Q, A],
    Or(P,Not(Q),Not(A)),
    Or(Not(Q),A,Not(P))
)

# Check if Horn
print(f'Is KB_horn a horn base? -> {KB_horn.is_horn()}')

print("\nKB_horn before conversion")
KB_horn.print_clauses()

# Converts horn clauses as implications
print("KB_horn after conversion")
KB_horn.horn_to_implication()
KB_horn.print_clauses()

Is KB_horn a horn base? -> True

KB_horn before conversion


KB_horn after conversion


### Forward chaining

Forward chaining is an automatic inference technique that starts from known facts and applies logical rules to derive new facts. It iteratively checks which rules have premises satisfied by the current facts, adding their conclusions to the knowledge base until no further inferences can be made. This method is typically used with knowledge bases expressed as Horn clauses.

Forward chaining works if and only if the following requirements are satisfied:

* The clauses in the KB are horn clauses
* Modus ponens is the main inference rule under the engine
* To apply modus ponens, all horn clauses in the KB must be expressed as implications
* All the knonw facts in the KB must be positive

In [18]:
"""
    Forward chaining inference algorithm.
    knownledge_base (sympy symbols): All clauses in the KB must be horn clauses
"""
def forward_chaining(knowledge_base):
    if not knowledge_base.is_horn():
        print("⚠️: The knowledge is a non-horn KB!")
        return None

    knowledge_base.horn_to_implication()    
    
    KB_changed = True
    clauses = knowledge_base.knowledge_base
    inferenced_rules = []
    
    while KB_changed:
        KB_changed = False
        facts = knowledge_base.extract_facts()
        for clause in clauses:
            if isinstance(clause, Implies):
                if modus_ponens(clause, facts):
                    conclusion = clause.args[1]
                    if conclusion not in facts:
                        knowledge_base.add_clauses(conclusion)
                        inferenced_rules.append(conclusion)
                        KB_changed = True
    return inferenced_rules

# Modus ponens inference rule
def modus_ponens(implication, facts):
    if not isinstance(implication, Implies):
        return None

    premises = set(implication.args[0].atoms())
    if premises.issubset(facts):
        return True
    return False

# Main
        
P, Q, M, L, A, B = symbols("P Q M L A B")

KB = KB_class(
        [P, Q, M, L, A, B], # symbols
        # clauses
        Implies(P,Q),
        Or(Not(L),Not(M),P),
        Implies(And(B,L),M),
        Implies(And(A,P),L),
        Implies(And(A,B),L),
        A,
        B
    )

print("Original KB")
KB.print_clauses()
print()

print("KB after forward chaining")
new_facts = forward_chaining(KB)
KB.print_clauses()
print(f"Inferenced facts: {new_facts}")

Original KB



KB after forward chaining


Inferenced facts: [L, M, P, Q]


### Backward chaining

Backward chaining is a goal-driven inference method that starts from a query and recursively checks if it can be derived from known facts by proving the premises of matching implication rules.

Like forward chaining, also in this case the algorithm works if the clauses of the KB are horn clauses expressed as implications, and modus ponens is the main inference rule. In addition, you have to pay attention to the loops during the inference.

Suppose that our KB is composed by:

```python
KB = (
    ¬rain OR wetGrass      # pure horn clause
    srpinkler -> wetGrass
    wetGrass -> slippery
    rain
)
```

We want check if a goal $\alpha$, can be infered by the KB. For instance, our goal could be $\alpha=\texttt{slippery}$.
In order to check the inference the backward chaining would proceed following theese steps:

1. Goal: slippery — not a known fact.
2. Find rules with conclusion slippery: wetGrass → slippery.
3. New subgoal: wetGrass.
4. Find rules with conclusion wetGrass: ¬rain ∨ wetGrass (rain → wetGrass), sprinkler → wetGrass, srpinkler → wetGrass.
5. Check if rain is a known fact.
6. rain is known → infer wetGrass.
7. From wetGrass and wetGrass → slippery → infer slippery.

Conclusion: `slippery` is inferred by backward chaining.

In [19]:
from sympy import Implies, And, Symbol

"""
    Backward chaining inference algorithm

    @params
        goal (sympy Symbol): sympy Symbol object which express a fact 
        knowledge_base (KB_class): Knoweledge base
        visited (bool): This parameter is useful for recursion
"""
def backward_chaining(goal, knowledge_base, visited=None):
    # If KB is not horn, backward is useless
    if knowledge_base.is_horn():
        knowledge_base.horn_to_implication()
    else:
        return False
    
    if visited is None:
        visited = set()

    # In order to avoid loops
    if goal in visited:
        return False
    
    visited.add(goal)

    # Check if the goal is already a known fact
    facts = knowledge_base.extract_facts()
    if goal in facts:
        return True

    # Check if goal is a conquent for any implication
    for clause in knowledge_base.knowledge_base:
        if isinstance(clause, Implies):
            conclusion = clause.args[1]
            premises = clause.args[0]
            if conclusion == goal:
                if isinstance(premises, Symbol):
                    if backward_chaining(premises, knowledge_base, visited):
                        return True
                elif isinstance(premises, And):
                    if all(backward_chaining(p, knowledge_base, visited) for p in premises.args):
                        return True
    return False

# Usage

rain, wetGrass, slippery, srpinkler = symbols("rain wetGrass slippery sprinkler")

KB = KB_class(
    [rain, wetGrass, slippery, srpinkler],
    Or(Not(rain), wetGrass),
    Implies(srpinkler, wetGrass),
    Implies(wetGrass, slippery),
    rain
)

query = slippery

print("Is alpha infered by KB?")
backward_chaining(query, KB)

Is alpha infered by KB?


True

## External references

[Sympy docs](https://docs.sympy.org/latest/tutorials/index.html)

[Propositional logic by *Geeks for Geeks*](https://www.geeksforgeeks.org/proposition-logic/)

ℹ️: A python source code which contains the implementation of all useful functions seen in this notebook is available in this project folder, in order to plug theese tools in others projects.

## Author

*Emilio Garzia, 2025*